In [2]:
import tensorflow as tf
from tensorflow import keras

import os
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0'
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam


In [3]:
dataset_path = '/Users/mohammadnur/Downloads/Imagedata'

In [4]:
# Load dataset with resizing images to the same dimensions
def load_dataset(dataset_path):
    images = []
    labels = []
    label_names = os.listdir(dataset_path)
    for index, name in enumerate(tqdm(label_names)):
        folder_path = os.path.join(dataset_path, name)
        for image_name in os.listdir(folder_path):
            image_path = os.path.join(folder_path, image_name)
            image = cv2.imread(image_path)
            if image is not None:  # Check if the image was successfully loaded
                image = cv2.resize(image, (200, 200))  # Resize images to 224x224 pixels
                images.append(image)
                labels.append(index)
            else:
                print(f"Warning: Could not load image {image_path}")
    images = np.array(images, dtype='float32') / 255.0  # Normalize images
    labels = np.array(labels)
    labels = to_categorical(labels, num_classes=len(label_names))
    return images, labels, label_names

images, labels, label_names = load_dataset(dataset_path)


  7%|▋         | 5/76 [00:00<00:02, 25.63it/s]

 11%|█         | 8/76 [00:00<00:06, 11.16it/s]

 13%|█▎        | 10/76 [00:00<00:06, 10.50it/s]

 18%|█▊        | 14/76 [00:01<00:08,  7.75it/s]

 21%|██        | 16/76 [00:01<00:07,  7.97it/s]

...

 83%|████████▎ | 63/76 [00:13<00:05,  2.55it/s]

 87%|████████▋ | 66/76 [00:13<00:02,  3.50it/s]

 88%|████████▊ | 67/76 [00:14<00:03,  2.92it/s]

 95%|█████████▍| 72/76 [00:16<00:01,  2.35it/s]

100%|██████████| 76/76 [00:17<00:00,  4.43it/s]


In [5]:
# Split dataset
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [7]:
import pickle

# Save pickled data
with open("/Users/mohammadnur/Documents/cvpr_23_24_spring/X_train.pickle", "wb") as pickle_out:
    pickle.dump(X_train, pickle_out)


with open("/Users/mohammadnur/Documents/cvpr_23_24_spring/Y_train.pickle", "wb") as pickle_out:
    pickle.dump(y_train, pickle_out)

# with open("/Users/mohammadnur/Documents/cvpr_23_24_spring/X_valid.pickle", "wb") as pickle_out:
#     pickle.dump(X_valid, pickle_out)

# with open("/Users/mohammadnur/Documents/cvpr_23_24_spring/Y_valid.pickle", "wb") as pickle_out:
#     pickle.dump(Y_valid, pickle_out)

with open("/Users/mohammadnur/Documents/cvpr_23_24_spring/X_test.pickle", "wb") as pickle_out:
    pickle.dump(X_test, pickle_out)

with open("/Users/mohammadnur/Documents/cvpr_23_24_spring/Y_test.pickle", "wb") as pickle_out:
    pickle.dump(y_test, pickle_out)

NameError: name 'y_test' is not defined

In [5]:


# def build_model(num_classes):
#     model = models.Sequential([
#         layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
#         layers.MaxPooling2D((2, 2)),
#         layers.Conv2D(64, (3, 3), activation='relu'),
#         layers.MaxPooling2D((2, 2)),
#         layers.Conv2D(128, (3, 3), activation='relu'),
#         layers.Flatten(),
#         layers.Dense(256, activation='relu'),
#         layers.Dense(num_classes, activation='softmax'),
#     ])
#     return model

# model = build_model(num_classes=len(label_names))
# model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])


In [6]:
# Build CNN model
def build_model(num_classes):
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=(200, 200, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu'),
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.Dense(num_classes, activation='softmax'),
    ])
    return model

model = build_model(num_classes=len(label_names))
model.compile(optimizer=Adam(learning_rate=0.01), loss='categorical_crossentropy', metrics=['accuracy'])


/opt/homebrew/anaconda3/lib/python3.11/site-packages/keras/src/layers/convolutional/base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [7]:
# Train model
history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=32)


Epoch 1/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 184s 11s/step - accuracy: 0.0359 - loss: 149.3205 - val_accuracy: 0.0606 - val_loss: 4.0165
Epoch 2/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 130s 7s/step - accuracy: 0.0572 - loss: 3.9413 - val_accuracy: 0.0606 - val_loss: 3.7271
Epoch 3/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 131s 7s/step - accuracy: 0.0579 - loss: 3.7418 - val_accuracy: 0.0606 - val_loss: 3.6654
Epoch 4/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 106s 6s/step - accuracy: 0.0483 - loss: 3.7084 - val_accuracy: 0.0303 - val_loss: 3.6924
Epoch 5/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 1118s 69s/step - accuracy: 0.0602 - loss: 3.6939 - val_accuracy: 0.0606 - val_loss: 3.6983
Epoch 6/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 354s 21s/step - accuracy: 0.0533 - loss: 3.7109 - val_accuracy: 0.0606 - val_loss: 3.6748
Epoch 7/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 449s 27s/step - accuracy: 0.0488 - loss: 3.7093 - val_accuracy: 0.0303 - val_loss: 3.6924
Epoch 8/20
17/17 ━━━━━━━━━━━━━━━━━━━━ 4216s 263s/step - accuracy: 0.0554 - loss: 3.6803 - val_accuracy: 

In [9]:
import cv2 
  
  
# define a video capture object 
vid = cv2.VideoCapture(0) 
  
while(True): 
      
    # Capture the video frame 
    # by frame 
    ret, frame = vid.read() 
  
    # Display the resulting frame 
    cv2.imshow('frame', frame) 
      
    # the 'q' button is set as the 
    # quitting button you may use any 
    # desired button of your choice 
    if cv2.waitKey(1) & 0xFF == ord('q'): 
        break
  
# After the loop release the cap object 
vid.release() 
# Destroy all the windows 
cv2.destroyAllWindows() 

KeyboardInterrupt: 

In [10]:
import cv2
import numpy as np

def real_time_recognition(model, label_names):
    # Initialize the webcam (use 0 as the parameter to select the default webcam)
    cap = cv2.VideoCapture(0)
    
    # Check if the webcam is opened correctly
    if not cap.isOpened():
        raise IOError("Cannot open webcam")
    
    while True:
        ret, frame = cap.read()  # Read a frame from the webcam
        if not ret:
            break
        
        # Preprocess the frame for prediction (resize, expand dimensions, normalize)
        face = cv2.resize(frame, (224, 224))  # Assuming your model expects 224x224 inputs
        face = np.expand_dims(face, axis=0)  # Add batch dimension
        face = face / 255.0  # Normalize pixel values to [0, 1] as during training
        
        prediction = model.predict(face)  # Make prediction
        predicted_class = label_names[np.argmax(prediction)]  # Get the name of the predicted class
        
        # Display the predicted class on the frame
        cv2.putText(frame, predicted_class, (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
        
        # Show the frame
        cv2.imshow('Real-time Face Recognition', frame)
        
        # Break the loop when the 'q' key is pressed
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    # Release the webcam and close all OpenCV windows
    cap.release()
    cv2.destroyAllWindows()

# Assuming 'model' is your trained model and 'label_names' is a list of class names
real_time_recognition(model, label_names)


ValueError: Exception encountered when calling Sequential.call().

[1mInput 0 of layer "dense" is incompatible with the layer: expected axis -1 of input shape to have value 270848, but received input with shape (1, 346112)[0m

Arguments received by Sequential.call():
  • inputs=tf.Tensor(shape=(1, 224, 224, 3), dtype=float32)
  • training=False
  • mask=None